In [38]:
import numpy as np
from itertools import combinations

In [39]:
%run std_input_matrix.ipynb


Vincolo numero 0
Inserire coefficiente numero 0
Inserire coefficiente numero 1
Inserire termine noto: 

Vincolo numero 1
Inserire coefficiente numero 0
Inserire coefficiente numero 1
Inserire termine noto: 

Vincolo numero 2
Inserire coefficiente numero 0
Inserire coefficiente numero 1
Inserire termine noto: 

Vincolo numero 3
Inserire coefficiente numero 0
Inserire coefficiente numero 1
Inserire termine noto: 
[[1. 1.]
 [1. 0.]
 [0. 1.]
 [1. 2.]]
[[12.]
 [10.]
 [ 6.]
 [16.]]
[[1. 1. 1. 0. 0. 0.]
 [1. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 1. 0.]
 [1. 2. 0. 0. 0. 1.]]
[[12.]
 [10.]
 [ 6.]
 [16.]]


Un’ulteriore implementazione del metodo del simplesso è nota come metodo
del simplesso tabellare. 
Pur essendo meno efficiente in termini di complessità computazionale rispetto al simplesso revisionato, il simplesso tabellare presenta il vantaggio di riportare in forma compatta tutti i dati necessari alla computazione in un’unica tabella, detta anche tableau.

Il simplesso tabellare aggiorna ad ogni iterazione la matrice:
$$B^{-1}[b|A] \in \mathbb{R}^{m \dot (n+1)}$$
le cui colonne sono gli $n+1$ vettori $B^{−1}b, B^{−1}A_{1}, B^{−1}A_{2},..., B^{−1}A_{n}$. 

La colonna $B^{−1}b$ è detta **colonna zero** e contiene i valori assunti dalle variabili
**correntemente in base**.

Supponiamo che la variabile **entrante in base** sia $x_{j}$, mentre quella **uscente dalla base** sia $x_{B(l)}$. Allora, la j-ma colonna del tableau $B^{−1}A_{j}$ è detta **colonna di pivot**; la riga l-ma è detta riga di pivot e l’elemento all’incrocio della colonna e della riga di pivot è detto **elemento di pivot**. Si noti che l’elemento di pivot è $u_{l}$ ed è sempre positivo, perchè se $u \le 0$, ciò comporterebbe la terminazione dell’algoritmo con valore ottimo della funzione obiettivo pari a $-\inf$.

Le informazioni contenute nelle righe del tableau hanno, invece, la seguente interpretazione: 

Dati i vincoli del problema $b = Ax$ e data la matrice corrente di base $B$, i vincoli possono
anche essere equivalentemente espressi come:
$$B^{-1}b = B^{-1}Ax$$
che corrisponde esattamente all'informazione contenuta nelle righe del tableau.

Alla fine di ogni iterazione del simplesso tabellare è necessario aggiornare il tableau $B^{−1}[b|A]$ per ottenere $\bar{B}^{−1}[b|A]$, dove $\bar{B}$ è la nuova matrice di base.

Siffatto aggiornamento è realizzato applicando al tableau una serie di operazioni di riga elementari atte a trasformare la colonna j-ma del tableau nel vettore unitario $e_{l}$, il vettore, cioè, contenete tutti zeri ed un unico uno nella riga l-ma.

Per quanto riguarda, invece, la determinazione del valore di $\theta*$ e della variabile $x_{B(l)}$ uscente dalla base, basta controllare i valori $\frac{x_{B(i)}}{ui}$ dati dal rapporto fra l'i-ma entrata della colonna zero e l'i-ma entrata della colonna di pivot, tali che ui sia positivo: il più piccolo di tali rapporti fornisce il valore da assegnare a $\theta*$ e determina anche la variabile $x_{B(l)}$.

Al tableau tipicamente è aggiunta anche una riga zero, il cui primo elemento situato nell’angolo in alto a sinistra corrisponde a:
$$−c'_{B}B^{-1}b$$ 
cioè al valore cambiato di segno che la funzione obiettivo assume in corrispondenza della soluzione xB correntemente in base. 

I restanti $n$ elementi della riga zero corrispondono al vettore dei costi ridotti:
$$c' - c'_{B}B^{-1}A$$

Guardando i costi ridotti contenuti nella riga zero, l’indice j della variabile entrante in base è immediatamente determinabile e, una volta determinato, la riga zero subisce anch’essa le opportune trasformazioni affinchè $c_{j}$ sia reso nullo.

In [40]:
print('vettori base:\n', vettori_base)
print('vettori fuori base:\n', vettori_fuori_base)

vettori base:
 [0, 1, 2, 3]
vettori fuori base:
 [4, 5]


In [41]:
list_coff

[-3, -5, 0, 0, 0, 0]

In [42]:
# creo la prima colonna del tableau:
# - elemento 0 (vertice in alto a sinistra): valore della funzione obiettivo
#                                            cambiato di segno valutata in xb;
# - gli altri elementi: valore relativo ad ogni variabile in base della soluzione xb
# creo la prima riga del tableau:
# - elemento 0: come sopra;
# - gli altri elementi:    cbar_j = c'_j - c'b * B^(-1) * A_j
#                      dove: 
#                       - cbar_j è il j-esimo costo ridotto;
#                       - c'_j coefficiente j-esimo della funzione obiettivo;
#                       - c'b è il vettore dei coefficienti della funzione obiettivo
#                         relativi slle variabili di base corrente;
#                       - B è la matrice di base;
#                       - A_j è la j-esima colonna di A.

# elemento 0
zero_elem = -np.dot(np.array([list_coff[i] for i in vettori_base]) , xb)

# altri elementi

# lista dei coefficienti dei vettori base
cprimeb = [list_coff[i] for i in vettori_base]
print(cprimeb)
# inizializzo lista di tutti i costi ridotti
c_j_list = [] 

for j in range(len(list_coff)):
    # calcolo dei costi:
    # c_j = cj - c'b * B(-1) * Aj
    c_j = list_coff[j] - np.dot(np.dot(cprimeb, np.linalg.inv(sub_matrix)), matrix_A[:,j])
    c_j_list.append(c_j)
# lista in cui salviamo l'indice delle variabili relative ad un costo ridotto negativo
neg_c_j_ind = np.where(np.array(c_j_list)<0)

[-3, -5, 0, 0]


In [43]:
c_j_list = [round(i, 5) for i in c_j_list]
for i in range(len(c_j_list)):
    if c_j_list[i]==-0.0 and i in neg_c_j_ind:
        neg_c_j_ind.remove(i)
        c_j_list[i] = -c_j_list[i]   
c_j_list

[0.0, 0.0, 0.0, 0.0, -1.0, 3.0]

In [44]:
first_row = list(zero_elem) + c_j_list
tableau = []
for i, j in zip(matrix_Aprime, xb):
    row = list(j) + list(i)
    print(row)
    tableau.append(row)
tableau.insert(0, first_row)

[4.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0]
[6.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0]
[2.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0]
[6.0, 1.0, 2.0, 0.0, 0.0, 0.0, 1.0]


In [45]:
tableau = np.array(tableau)
tableau

array([[42.,  0.,  0.,  0.,  0., -1.,  3.],
       [ 4.,  1.,  1.,  1.,  0.,  0.,  0.],
       [ 6.,  1.,  0.,  0.,  1.,  0.,  0.],
       [ 2.,  0.,  1.,  0.,  0.,  1.,  0.],
       [ 6.,  1.,  2.,  0.,  0.,  0.,  1.]])

In [46]:
c_j_list = np.array(c_j_list)
flag_2 = (c_j_list>0).all()

while flag_2==False:
    print(neg_c_j_ind)
    # scegliamo l'indice più piccolo tra le variabili che hanno costo ridotto negativo
    min_ind = np.min(neg_c_j_ind) # REGOLA DI BLAND
    # calcolo u = B(-1) * A(indice minimo scelto)
    u = tableau[1:,neg_c_j_ind[0]+1]
    print('u: ', u)
    print('xb: ', xb)
    # indice della riga di pivot
    # seleziono min {x_i / u_i} -> indice della variabile uscente
    xb_u_list = []
    for xb_i, u_i in zip(xb, u):
        xb_u_list.append(xb_i/u_i)
    out_xbu = np.min([w for w in xb_u_list if w>0])
    out_ind = np.where(np.array(xb_u_list)==out_xbu)[0][0]
    print('out_ind: ', out_ind)
    # operazioni elementari tra righe per trasformare 
    # u -> e_j (j-esima comp della matrice identità)
    # r_p = r_p / r_p[neg_c_j_ind[0]+1]
    tableau[out_ind+1] = tableau[out_ind+1] / tableau[out_ind+1][neg_c_j_ind[0]+1]
    # per ogni riga r_j del tableau - esclusa quella di pivot (r_p)
    # r_j = r_j - r_j[neg_c_j_ind[0]+1] / r_p[neg_c_j_ind[0]+1]
    for j in range(tableau.shape[0]):
        if j==out_ind+1:
            continue
        tableau[j]= tableau[j] - tableau[j][neg_c_j_ind[0]+1] / tableau[out_ind+1][neg_c_j_ind[0]+1]
    neg_c_j_ind = np.where(tableau[0,1:]<0)[0]
    print(tableau)
    xb=tableau[1:,0]
    print('new_xb: ', xb)
    if (tableau[0,1:]>=0).all():
        flag_2 = True


(array([4], dtype=int64),)
u:  [[0.]
 [0.]
 [1.]
 [0.]]
xb:  [[4.]
 [6.]
 [2.]
 [6.]]
out_ind:  2
[[43.  1.  1.  1.  1.  0.  4.]
 [ 4.  1.  1.  1.  0.  0.  0.]
 [ 6.  1.  0.  0.  1.  0.  0.]
 [ 2.  0.  1.  0.  0.  1.  0.]
 [ 6.  1.  2.  0.  0.  0.  1.]]
new_xb:  [4. 6. 2. 6.]


<ipython-input-46-25e03a6e58cf>:16: RuntimeWarning: divide by zero encountered in true_divide
  xb_u_list.append(xb_i/u_i)


In [47]:
tableau

array([[43.,  1.,  1.,  1.,  1.,  0.,  4.],
       [ 4.,  1.,  1.,  1.,  0.,  0.,  0.],
       [ 6.,  1.,  0.,  0.,  1.,  0.,  0.],
       [ 2.,  0.,  1.,  0.,  0.,  1.,  0.],
       [ 6.,  1.,  2.,  0.,  0.,  0.,  1.]])

In [48]:
print('Costo ottimo: ', -tableau[0,0])


Costo ottimo:  -43.0
